## Without web search

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano"
)

In [7]:
import os
from langchain_ibm import ChatWatsonx

parameters = {
    "temperature": 0.5,
    "max_tokens": 100,
}

model = ChatWatsonx(
    model_id="meta-llama/llama-4-maverick-17b-128e-instruct-fp8",
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    params=parameters,
)

agent = create_agent(model=model)

In [11]:
from langchain.messages import HumanMessage

question = HumanMessage(content="How up to date is your training knowledge?")

response = agent.invoke(
    {"messages": [question]}
)

In [12]:
print(response['messages'][-1].content)

My training data goes up to around September 2023. I don’t have real-time access to new information unless a browsing tool is enabled in the environment. 

If you need the latest facts (post-September 2023), you can enable browsing or share sources, and I can help summarize or verify them. For anything up to that cutoff, I can generally help with accuracy and context.


## Add web search tool

In [13]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search.invoke("Who is the current mayor of San Francisco?")

{'query': 'Who is the current mayor of San Francisco?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Mayor_of_San_Francisco',
   'title': 'Mayor of San Francisco - Wikipedia',
   'content': 'The current mayor is Democrat Daniel Lurie.',
   'score': 0.9994253,
   'raw_content': None},
  {'url': 'https://apnews.com/article/san-francisco-new-mayor-liberal-city-81ea0a7b37af6cbb68aea7ef5cc6a4f0',
   'title': "San Francisco's new mayor is starting to unite the fractured city",
   'content': 'San Francisco Mayor Daniel Lurie, a political newcomer and Levi Strauss heir, has marked his first 100 days with a hands-on, business-friendly approach.',
   'score': 0.9993538,
   'raw_content': None},
  {'url': 'https://www.sf.gov/departments--office-mayor',
   'title': 'Office of the Mayor - SF.gov',
   'content': 'Daniel Lurie is the 46th Mayor of the City and County of San Francisco.',
   'score': 0.99620515,
   'raw_content': None

In [14]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search]
)

question = HumanMessage(content="Who is the current mayor of San Francisco?")

response = agent.invoke(
    {"messages": [question]}
)

In [15]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='Who is the current mayor of San Francisco?', additional_kwargs={}, response_metadata={}, id='79a8c9f2-8273-4de3-a0fa-9b54a22ceeb0'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 350, 'prompt_tokens': 133, 'total_tokens': 483, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpKqwccif4JDZHGPZ9TXxIVXA9NW5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b42b8-2bd8-7f32-a95e-4f96585b6526-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'Who is the current mayor of San Francisco'}, 'id': 'call_49MWX7HL3MUHJeSPByDAm6ea', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, '

trace: https://smith.langchain.com/public/59432173-0dd6-49e8-9964-b16be6048426/r